In [65]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os

import wfdb

In [66]:
record = wfdb.rdrecord('24h-emgcg/s0352-07060702', channels=[0, 1])
record.p_signal *= 1000  # Convert from uV to mV
wfdb.plot_wfdb(record=record, title='24h ecg emg')
print(record.__dict__)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/macad/Documents/University/Thesis/ECG Processing/elderly-stroke/24h-emgcg/s0352-07060702.hea'

In [ ]:
folder_path = '24h-emgcg/'

record_files = [f for f in os.listdir(folder_path) if f.endswith('.hea')]
records_samples = []

for record_file in record_files:
    record_name = os.path.splitext(record_file)[0]
    record_header = wfdb.rdheader(os.path.join(folder_path, record_name))
    num_samples = record_header.sig_len
    records_samples.append([record_name, num_samples])

# Now records_samples contains a list of [record_name, num_samples] for each record
for record, samples in records_samples:
    print(f"Record '{record}' has {samples} samples.")


Record 's0218-05110112' has 30260496 samples.
Record 's0194-05071409' has 39549486 samples.
Record 's0064-05081806' has 57574788 samples.
Record 's0321-07022204' has 21912 samples.
Record 's0067-05040511' has 49570746 samples.
Record 's0132-04102802' has 46897408 samples.
Record 's0242-06012407' has 37140432 samples.
Record 's0030-04051908' has 55066318 samples.
Record 's0067-05040510' has 8035950 samples.
Record 's0321-07022205' has 27890484 samples.
Record 's0194-05071408' has 43629450 samples.
Record 's0221-06082201' has 16404 samples.
Record 's0218-05110113' has 51666246 samples.
Record 's0221-06082203' has 6888 samples.
Record 's0064-05081805' has 25282776 samples.
Record 's0172-05071906' has 40709484 samples.
Record 's0132-04102801' has 26168545 samples.
Record 's0243-06031609' has 33042288 samples.
Record 's0232-06060108' has 26105460 samples.
Record 's0242-06012405' has 45681954 samples.
Record 's0244-06041304' has 54407868 samples.
Record 's0321-07022206' has 55360422 samples.

In [ ]:
import wfdb
import os

# Define the folder path containing the ECG records
folder_path = '24h-emgcg/'

# List all .hea files in the folder
record_files = [f for f in os.listdir(folder_path) if f.endswith('.hea')]

# Initialize an array to hold the names of records missing .dat files
records_missing_dat = []

records_exist = []

for record_file in record_files:
    # Extract the base name of the record (without extension)
    record_name = os.path.splitext(record_file)[0]
    
    # Check if the corresponding .dat file exists
    if not os.path.exists(os.path.join(folder_path, f"{record_name}.dat")):
        records_missing_dat.append(record_name)
        continue  # Skip to the next record
    
    # Define the path for the output .p_signal file
    output_file_path = os.path.join(folder_path, f"{record_name}_p_signal.txt")
    
    # Check if the .p_signal file already exists
    if os.path.exists(output_file_path):
        records_exist.append(record_file)
        continue  # Skip to the next record if .p_signal file exists
    
    # Read the specified channels (0 and 1) from the record
    try:
        record = wfdb.rdrecord(os.path.join(folder_path, record_name), channels=[0, 1])
    except FileNotFoundError:
        # This block catches the case where the .hea file exists without its .dat file,
        # although this should be primarily handled by the earlier .dat check
        records_missing_dat.append(record_name)
        continue
    
    # Extract the signals for channels 0 and 1
    signals = record.p_signal
    
    # Write the signals to the output file
    with open(output_file_path, 'w') as file:
        for signal in signals:
            file.write(f"{signal[0]}\t{signal[1]}\n")

# Output the records that were skipped due to missing .dat files
print("Conversion completed. The following records were skipped due to missing .dat files:")
print(records_missing_dat)

print("amount built: ")
print(len(records_exist))

Conversion completed. The following records were skipped due to missing .dat files:
[]
amount built: 
143


In [64]:
folder_path = "24h-emgcg"

# Define file extensions to delete
extensions_to_delete = ['.dat', '.hea', '.marker']

# Function to delete files with given extensions
def delete_files_with_extensions(folder_path, extensions):
    for ext in extensions:
        file_pattern = os.path.join(folder_path, f'*{ext}')
        files = glob.glob(file_pattern)
        for file in files:
            try:
                os.remove(file)
                print(f"Deleted: {file}")
            except OSError as e:
                print(f"Error: {file} - {e.strerror}")

# Delete files
delete_files_with_extensions(folder_path, extensions_to_delete)


Deleted: 24h-emgcg/s0121-04042802.dat
Deleted: 24h-emgcg/s0184-05091310.dat
Deleted: 24h-emgcg/s0030-04051907.dat
Deleted: 24h-emgcg/s0183-05042809.dat
Deleted: 24h-emgcg/s0183-05042808.dat
Deleted: 24h-emgcg/s0207-05090814.dat
Deleted: 24h-emgcg/s0215-06030711.dat
Deleted: 24h-emgcg/s0184-05091311.dat
Deleted: 24h-emgcg/s0225-06022112.dat
Deleted: 24h-emgcg/s0121-04042801.dat
Deleted: 24h-emgcg/s0172-05071908.dat
Deleted: 24h-emgcg/s0067-05040508.dat
Deleted: 24h-emgcg/s0044-05062806.dat
Deleted: 24h-emgcg/s0200-05072105.dat
Deleted: 24h-emgcg/s0200-05072104.dat
Deleted: 24h-emgcg/s0044-05062807.dat
Deleted: 24h-emgcg/s0215-06030712.dat
Deleted: 24h-emgcg/s0225-06022111.dat
Deleted: 24h-emgcg/s0067-05040509.dat
Deleted: 24h-emgcg/s0214-05110807.dat
Deleted: 24h-emgcg/s0154-05050307.dat
Deleted: 24h-emgcg/s0160-04052807.dat
Deleted: 24h-emgcg/s0221-06082208.dat
Deleted: 24h-emgcg/s0228-06011209.dat
Deleted: 24h-emgcg/s0210-05092011.dat
Deleted: 24h-emgcg/s0221-06082209.dat
Deleted: 24h

In [74]:

# Define the PhysioNet directory for the dataset
pn_dir = 'cves'
pn_dir_full = 'cves/1.0.0/data/24h-electromyography'

# Local directory to save the converted .p_signal files
local_save_dir = '24h-emgcg/'

# Ensure the local save directory exists
os.makedirs(local_save_dir, exist_ok=True)

# Get the list of records from the specified PhysioNet directory
record_names = wfdb.get_record_list(pn_dir)

# Get list of ecg-emg record from the list above by cut off the prefix
record_ecgmg = [record.split('/')[2] for record in record_names if record.startswith("data/24h-electromyography/")]

for record_name in record_ecgmg:
    # Define the path for the expected .p_signal output file
    output_file_path = os.path.join(local_save_dir, f"{record_name}_p_signal.txt")
    
    # Check if the .p_signal file already exists
    if os.path.exists(output_file_path):
        print(f"File {output_file_path} already exists. Skipping...")
        continue

    # Read the specified channels (0 and 1) from the record directly from PhysioNet
    # Note: This step requires an internet connection and access permissions to the data
    record = wfdb.rdrecord(record_name, pn_dir=pn_dir_full, channels=[0, 1])

    # Extract the signals for channels 0 and 1
    signals = record.p_signal

    # Write the signals to the output file
    with open(output_file_path, 'w') as file:
        for signal in signals:
            # Write each signal sample to the file, separated by a tab
            file.write(f"{signal[0]}\t{signal[1]}\n")

print("Conversion completed. Check the local directory for the .p_signal files.")


File 24h-emgcg/s0030-04051907_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0030-04051908_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0044-05062806_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0044-05062807_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0064-05081805_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0064-05081806_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0067-05040508_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0067-05040509_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0067-05040510_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0067-05040511_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0068-05022417_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0068-05022418_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0121-04042801_p_signal.txt already exists. Skipping...
File 24h-emgcg/s0121-04042802_p_signal.txt already exists. Skipping...
File 2